In [8]:
#SSDクラスを作成する
import myNetwork
import torch.nn as nn
import torch.nn.functional as F
from Detect import Detect

class SSD(nn.Module):
    
    def __init__(self, phase, cfg):
        super(SSD, self).__init__()
        
        self.phase = phase  #train or inference
        self.num_classes = cfg["num_classes"] #クラス数21
        
        #SSDのネットワークを作る
        self.vgg = myNetwork.make_vgg()
        self.extras = myNetwork.make_extras()
        self.L2Norm = myNetwork.L2Norm()
        self.loc, self.conf = myNetwork.make_loc_conf(
                 cfg["num_classes"], cfg["bbox_aspect_num"])
        
        #DBox作成
        dbox = myNetwork.DBox(cfg)
        self.dbox_list = dbox.make_dbox_list()
        
        #推論時はクラス「Detect]を用意
        if phase == 'inference':
            self.detect = Detect()
            
    def forward(self, x):
        sources = list() #locとconfへの入力source1~6を格納
        loc = list()     #locの出力を格納
        conf = list()    #confの出力を格納
        
        #vggのconv4_3まで計算する。
        for k in range(23):
            x = self.vgg[k](x)
        
        #conv4_3の出力をL2Norm層に入力し、source1を作成、sourcesに追加
        source1 = self.L2Norm(x)
        sources.append(source1)
        
        #vggを最後まで計算し、source2を作成, sourcesに追加
        for k in range(23, len(self.vgg)):
            x = self.vgg[k](x)
        
        sources.append(x)
        
        #extrasのconvとReLUを計算
        #source3~6をsourcesに追加
        for k, v in enumerate(self.extras):
            x = F.relu(v(x), inplace=True)
            if k % 2 == 1:       #conv->ReLU->conv->ReLUをしたらsourcesに追加
                sources.append(x) 
            
        #source1~6に、それぞれ対応する畳み込みを一回ずつ適応する。
        #zipでforループの複数のリストの要素を取得
        #source1~6まであるので、6回のループが回る。
        for (x, l, c) in zip(sources, self.loc, self.conf):
            #Permuteは要素の順番を入れ替え
            loc.append(l(x).permute(0,2,3,1).contiguous())
            conf.append(c(x).permute(0,2,3,1).contiguous())
            #l(x)とc(x)で畳み込みを実行
            #l(x)の出力サイズは
            #[batch_num, 4*アスペクト比の種類数, featuremapの高さ,featuremapの幅]
            #c(x)の出力サイズは
            #[batch_num, 21*アスペクト比の種類数, featuremapの高さ,featuremapの幅]
            #sourceによって、アスペクト比の種類数が異なり、面倒なので順番を入れ替える
            #permuteで要素の順番を入れ替え
            #l(x)->[minbatch数, featuremap縦マス数, featuremapの横マス数, 4*アスペクト比の種類数]へ
            #c(x)->[minbatch数, featuremapの縦マス数, featuremapの横マス数, 21*アスペクト比の種類数]へ
            #(注釈)
            #torch.contiguous()はメモリ上で要素を連続的に配置し直す命令
            #あとでview関数を使用するが、そのためには対象の変数がメモリ上で
            #連続配置されている必要がある。
            
            
        #--------------ここから下はsourceという概念はない----------------
        #さらにlocとconfの形を変形
        #locのサイズは、torch.Size([batch_num, 34928])
        #confのサイズは、torch.Size([batch_num, 183372])になる
        loc = torch.cat([o.view(o.size(0), -1) for o in loc], 1)
        conf = torch.cat([o.view(o.size(0), -1) for o in conf], 1)
               #torch.view(*dim)のどこか1つだけ-1にすることができ、
               #-1で指定されたdimに関しては大きさが自動調整される。
        
        #さらにlocとconfの形を整える
        #locのサイズは、torch.Size([batch_num, 8732, 4])
        #confのサイズは、torch.Size(batch_num. 8732, 21)になる
        loc = loc.view(loc.size(0), -1, 4)
        conf = conf.view(conf.size(0), -1, self.num_classes)
        
        #最後に出力する
        output = (loc, conf, self.dbox_list)
        
        if self.phase == 'inference':   #推論時
            #クラス「Detect」のforwardを実行 
            #オーバーライド???
            #返り値のサイズはtorch.Size([batch_num, 21, 200, 5])
            return self.detect(output[0], output[1], output[2])
        
        else:#学習時
            return output
            #返り値は(loc, conf, dbox_list)のタプル